# Build pipeline with command function

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define `CommandComponent` using command() function
- Create `Pipeline` using component defined by command() function

**Motivations** - This notebook explains how to define `CommandComponent` via command function, then use command component to build pipeline. The command component is a fundamental construct of Azure Machine Learning pipeline. It can be used to run a task on a specified compute (either local or on the cloud). The command component accepts `Environment` to setup required infrastructure. You can define a `Command` to run on this infrastructure with `inputs`. You can reuse the same `Component` with different pipeline.  

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [1]:
#import required libraries
from azure.ml import MLClient, dsl, command
from azure.ml.entities import JobInput, JobOutput, Environment
from azure.ml.entities._assets.environment import BuildContext
import piprunpkg

## 1.2. Configure credential

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [2]:
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token('https://management.azure.com/.default')
except Exception as ex:
    # If exception happens when retrieve token, try exclude the failed credential like this then try again:
    # Exclude VSCode credential:
    # credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)
    raise Exception("Failed to retrieve a token from the included credentials due to the following exception, try to add `exclude_xxx_credential=True` to `DefaultAzureCredential` and try again.") from ex

EnvironmentCredential.get_token failed: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no managed identity endpoint found.
SharedTokenCacheCredential.get_token failed: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
VisualStudioCodeCredential.get_token failed: Failed to get Azure user details from Visual Studio Code.


## 1.3. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. 

In [3]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "d511f82f-71ba-49a4-8233-d7be8a3650f4",
        "resource_group": "mire2etesting",
        "workspace_name": "gewa_ws"
    }

    if client_config["subscription_id"].startswith('<'):
        print("please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <WORKSPACE_NAME> in notebook cell")
        raise ex
    else:  # write and reload from config file
        import json, os
        config_path = "../../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: D:\practice\azureml-examples\sdk\jobs\.azureml\config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000001F47A75B370>,
         subscription_id=d511f82f-71ba-49a4-8233-d7be8a3650f4,
         resource_group_name=mire2etesting,
         workspace_name=gewa_ws)


## 1.4. Retrieve or create an Azure Machine Learning compute target

In [4]:
# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
try:
    ml_client.compute.get(name=cluster_name)
except Exception:
    print('Creating a new compute target...')
    from azure.ml.entities import AmlCompute
    compute = AmlCompute(
        name=cluster_name,
        size="Standard_D2_v2",
        max_instances=2
    )
    ml_client.compute.begin_create_or_update(compute)

# 2. Define Command object via command function 
Use `command` function to create a `Command` object which can be used in `dsl.pipeline`


In [2]:
download_url = "https://azuremlexamples.blob.core.windows.net/datasets/iris.csv"
file_name = "data.csv"

environment = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
# 1. Create a command component to download a input data
download_data = command(
    name="download-input",
    display_name="Download Data",
    description="Download a input from remote URL and return it in output.",
    tags=dict(),
    command="curl ${{inputs.url}} > ${{outputs.output_folder}}/${{inputs.file_name}}",
    environment=environment,
    inputs=dict(
        url=download_url,
        file_name=file_name
    ),
    outputs=dict(
        output_folder=JobOutput(type="uri_folder", mode="rw_mount")
    )
)
# 2. Create a R script command to train data
train_data_with_r = command(
    name="train-data-with-R",
    display_name="Train Data with R",
    description="Train data with R.",
    tags=dict(),
    command="Rscript train.R --data_folder ${{inputs.iris}}/${{inputs.file_name}}",
    environment=Environment(build=BuildContext(path="docker_context")),
    code="./src",
    inputs=dict(
        iris=JobInput(type="uri_folder"),
        file_name=file_name,
    ),
    outputs={}
)
# 3. Create a command component to show inputs
show_data = command(
    name="show-data",
    display_name="Show Data",
    description="Show data in command line and return it in output.",
    tags=dict(),
    command="cat ${{inputs.input_folder}}/${{inputs.file_name}} && cp ${{inputs.input_folder}}/${{inputs.file_name}} ${{outputs.output_folder}}/${{inputs.file_name}}",
    environment=environment,
    inputs=dict(
        input_folder=JobInput(type="uri_folder"),
        file_name=file_name
    ),
    outputs=dict(
        output_folder=JobOutput(type="uri_folder")
    )
)

# 3. Basic pipeline job

## 3.1 Build pipeline

Build a pipeline with predefined commands.

In [3]:
compute = 'cpu-cluster'
@dsl.pipeline(
    description="The hello world pipeline job",
    tags={"owner": "sdkteam", "tag": "tagvalue"},
    default_compute=compute,
)
def pipeline_with_non_python_components(url, file_name):
    download_data_node = download_data(url=url, file_name=file_name)
    train_data_with_r(iris=download_data_node.outputs.output_folder, file_name=file_name)
    show_data_node = show_data(input_folder=download_data_node.outputs.output_folder, file_name=file_name)

    return {
        "output_file": show_data_node.outputs.output_folder
    }

pipeline = pipeline_with_non_python_components(download_url, file_name)
print(pipeline)

name: silver_whistle_ygsqsp8v27
display_name: pipeline_with_non_python_components
description: The hello world pipeline job
type: pipeline
inputs:
  url: https://azuremlexamples.blob.core.windows.net/datasets/iris.csv
  file_name: data.csv
outputs:
  output_file: null
tags:
  owner: sdkteam
  tag: tagvalue
compute: azureml:cpu-cluster
settings: {}
properties: {}
experiment_name: 1d_pipeline_with_non_python_components
jobs:
  download_data_node:
    $schema: '{}'
    type: command
    inputs:
      url: ${{parent.inputs.url}}
      file_name: ${{parent.inputs.file_name}}
    outputs:
      output_folder:
        mode: rw_mount
        type: uri_folder
    command: curl ${{inputs.url}} > ${{outputs.output_folder}}/${{inputs.file_name}}
    environment_variables: {}
    component:
      name: download-input
      display_name: Download Data
      description: Download a input from remote URL and return it in output.
      type: command
      inputs:
        url:
          type: string
   

In [5]:
from piprunpkg import run_pipeline, CuratedEnvironmentDefaultRunMode
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
output_root_dir = "./local-run-output"
run_pipeline(output_root_dir=output_root_dir, pipeline_job=pipeline, curated_env_default_run_mode=CuratedEnvironmentDefaultRunMode.CONTAINER)

INFO:executor:############################################download_data_node##############################################


NotADirectoryError: [WinError 267] The directory name is invalid

# 3.2 Submit pipeline job

In [ ]:

pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name="pipeline_samples")
pipeline_job
ml_client.jobs._runs_operations._operation.add

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)